<a href="https://colab.research.google.com/github/Fanusaez/7506R-2C2022-GRUPO13/blob/main/7506R_TP1_GRUPO13_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hOLA")

hOLA
